# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [105]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [106]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/Users/wyattshapiro/projects/courses/udacity_data_engineering/data_modeling_with_cassandra/project/src


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [107]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [108]:
# check the number of rows in your sample csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Insert into Apache Cassandra DB. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [109]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [110]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS song_event
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [111]:
try:
    session.set_keyspace('song_event')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [112]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

## Primary Key
## 1. Partition key: session_id
##    - Because this query is focused on investigating history during a session, 
##      it can be optimized by partitions across session_id.
## 2. Clustering columns: item_in_session
##    - In order to make each row unique, adding item_in_session is required.

## Create table
query_create = "CREATE TABLE IF NOT EXISTS song_session_history"
query_create = query_create + "(session_id int, item_in_session int, artist_name text, song_name text, song_length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query_create)
except Exception as e:
    print(e)                    

In [113]:
## Insert data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query_insert = "INSERT INTO song_session_history (session_id, item_in_session, artist_name, song_name, song_length)"
        query_insert = query_insert + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query_insert, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [114]:
## Verify that data was entered into the table
query_select = "select session_id, item_in_session, artist_name, song_name, song_length from song_session_history WHERE session_id = 338 and item_in_session = 4"

try:
    rows = session.execute(query_select)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.session_id, row.item_in_session, row.artist_name, row.song_name, row.song_length)

338 4 Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [115]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## Primary Key
## 1. Partition key: user_id
##    - Because this query is focused on investigating behavior of a user, 
##      it can be optimized by partitions across user_id.
## 2. Clustering columns: session_id, item_in_session
##    - In order to make each row unique, adding session_id and item_in_session is required.

## Create table
query_create = "CREATE TABLE IF NOT EXISTS user_session_history"
query_create = query_create + "(user_id int, session_id int, item_in_session int, artist_name text, song_name text, user_first_name text, user_last_name text, user_gender text, user_location text, PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query_create)
except Exception as e:
    print(e)        

In [116]:
## Insert data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query_insert = "INSERT INTO user_session_history (user_id, session_id, item_in_session, artist_name, song_name, user_first_name, user_last_name, user_gender, user_location)"
        query_insert = query_insert + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query_insert, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4], line[2], line[7]))
        

In [117]:
## Verify that data was entered into the table
query_select = "select user_id, session_id, item_in_session, artist_name, user_first_name, user_last_name from user_session_history WHERE user_id = 10 and session_id = 182"

try:
    rows = session.execute(query_select)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_id, row.session_id, row.item_in_session, row.artist_name, row.user_first_name, row.user_last_name)

10 182 0 Down To The Bone Sylvie Cruz
10 182 1 Three Drives Sylvie Cruz
10 182 2 Sebastien Tellier Sylvie Cruz
10 182 3 Lonnie Gordon Sylvie Cruz


In [118]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Primary Key
## 1. Partition key: session_id
##    - Because this query is focused on analyzing popularity of song across history, 
##      it can be optimized by partitions across song_name.
## 2. Clustering columns: user_id
##    - In order to make each row unique, adding user_id is required. 
##      This will ensure that users with duplicate names will not be dropped because id is unique.

## Create table
query_create = "CREATE TABLE IF NOT EXISTS songplay_history"
query_create = query_create + "(song_name text, user_id int, user_first_name text, user_last_name text, user_gender text, user_location text, PRIMARY KEY (song_name, user_id))"
try:
    session.execute(query_create)
except Exception as e:
    print(e)  

                    

In [119]:
## Insert data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query_insert = "INSERT INTO songplay_history (song_name, user_id, user_first_name, user_last_name, user_gender, user_location)"
        query_insert = query_insert + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query_insert, (line[9], int(line[10]), line[1], line[4], line[2], line[7]))

In [120]:
## Verify that data was entered into the table
query_select = "select song_name, user_id, user_first_name, user_last_name from songplay_history WHERE song_name = 'All Hands Against His Own'"

try:
    rows = session.execute(query_select)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song_name, row.user_id, row.user_first_name, row.user_last_name)

All Hands Against His Own 29 Jacqueline Lynch
All Hands Against His Own 80 Tegan Levine
All Hands Against His Own 95 Sara Johnson


### Drop the tables before closing out the sessions

In [121]:
query_delete = "drop table song_session_history"
try:
    rows = session.execute(query_delete)
except Exception as e:
    print(e)
    
query_delete = "drop table user_session_history"
try:
    rows = session.execute(query_delete)
except Exception as e:
    print(e)

query_delete = "drop table songplay_history"
try:
    rows = session.execute(query_delete)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [122]:
session.shutdown()
cluster.shutdown()